In [ ]:
import pandas as pd
import numpy as np
from xlrd import open_workbook

wb = open_workbook('initial_data.xls', formatting_info=True)
default_sheet = wb.sheet_by_index(0) 

color_to_string = {
    # Header
    (0, 0, 128): "",
    # No color
    None: "",
    # Yellow
    (255, 255, 204): "Air/Sea Disasters",
    # Tan
    (237, 218, 181): "Unprovoked Incidents",
    # Orange
    (255, 204, 153): "Provoked Incidents",
    # Blue
    (153, 204, 255): "Questionable Incidents",
    # Green
    (207, 238, 204): "Attacks on Boats",
    # Yellow (another "tone", but practically the same)
    (255, 255, 197): "Air/Sea Disasters"
}

In [93]:
def extract_incident_types(sheet):
    incident_types = []

    color_indexes = []
    unique_colors = set()
    color_idx = {}

    for row in range(sheet.nrows):
        c = sheet.cell(row, 1)
        xf = wb.xf_list[c.xf_index]
        color = wb.colour_map.get(xf.background.pattern_colour_index)

        color_indexes.append(color)

        if color not in unique_colors:
            color_idx[color] = row

        unique_colors.add(color)

        incident_type = color_to_string.get(color, "Unknown")  # Default to "Unknown"
        incident_types.append(incident_type)

    with open("colours.txt", 'w') as file:
        for item in color_indexes:
            file.write(f"{item}\n")  # Write each item on a new line    

    print(unique_colors)
    print(color_idx)

    return incident_types

In [94]:
incident_types = extract_incident_types(default_sheet)

data = []
for row in range(default_sheet.nrows):
    row_data = [default_sheet.cell(row, col).value for col in range(default_sheet.ncols)]
    data.append(row_data)

columns = [default_sheet.cell(0, col).value for col in range(default_sheet.ncols)]  # Use first row as header
df = pd.DataFrame(data, columns=columns)

df.head()

{None, (0, 0, 128), (255, 255, 197), (153, 204, 255), (237, 218, 181), (255, 255, 204), (207, 238, 204), (255, 204, 153)}
{(0, 0, 128): 0, None: 1, (255, 255, 204): 2, (237, 218, 181): 33, (255, 204, 153): 38, (153, 204, 255): 49, (207, 238, 204): 64, (255, 255, 197): 162}


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,,,,,,,,,,
0,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,45597.0,2024.0,Unprovoked,Hawaii,Maui,Sand Piles Waiehu Beach Park,Surfing,Kenji,M,61.0,...,,,,,,,,,,
3,45596.0,2024.0,Unprovoked,Australia,Queensland,Whitsunday Islands,Snorkeling,Not stated,F,57.0,...,,,,,,,,,,
4,45590.0,2024.0,Unprovoked,USA,Florida,"Bathtub Beach, Martin County",Surfing,Cole Taschman,M,27.0,...,,,,,,,,,,


In [95]:
df['Incident Type'] = incident_types

# Remove first two rows (redundant)
df = df.drop(index=0)
df = df.drop(index=1)
df = df.reset_index(drop=True)  # Reset the index

# Remove empty columns
df = df.loc[:, (df != '').any(axis=0)]
df.dropna(how='all', axis=1, inplace=True)

df.to_csv('first_cleaning.csv', index=False)

df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Source,pdf,href formula,href,Case Number,Case Number,original order,,,Incident Type
0,45597.0,2024.0,Unprovoked,Hawaii,Maui,Sand Piles Waiehu Beach Park,Surfing,Kenji,M,61.0,...,Glen Folkard: Independent: Fox News: Sky News ...,,,,,,,,,Air/Sea Disasters
1,45596.0,2024.0,Unprovoked,Australia,Queensland,Whitsunday Islands,Snorkeling,Not stated,F,57.0,...,Simon de Marchi: 9 News: 7 News: ABC News:,,,,,,,,,Air/Sea Disasters
2,45590.0,2024.0,Unprovoked,USA,Florida,"Bathtub Beach, Martin County",Surfing,Cole Taschman,M,27.0,...,Bob Myatt: Yardbarker: 12 on your side: WPTV: ...,,,,,,,,,Air/Sea Disasters
3,45579.0,2024.0,Unprovoked,Honduras,Tela,Atlantida,Swimming,Roberto Carlose Bu Mejia,M,38.0,...,Georgienne Bradley: Daily Mail UK: FTN News.com,,,,,,,,,Air/Sea Disasters
4,45576.0,2024.0,Unprovoked,USA,Florida,Brevard County Orlando,Surfing,Teddy Witteman,M,16.0,...,Todd SmithFlorida today: News 4:,,,,,,,,,Air/Sea Disasters
